In [3]:
import numpy as np
from PIL import Image
import os

# Part 1
- The approximation would consist of a grayscale image where the overall shape of the moon is captured but with no details. The variations in intensity across the moon's surface, such as craters and maria, would not be represented.

- The brightest part of the moon would likely be the most prominent feature in the approximation, as it contributes most to the variance. The darkest regions (the black background) would contribute least and hence would be approximated as uniform darkness.

The rank 1 approximation would attempt to capture the most dominant feature of the image. Since the image is grayscale, the dominant feature is typically related to the largest variance in intensity, which might associated with the brightest or the largest structure in the image.
<br>
<center>
<img src="./src/moon.png" width="800" /> 
</center>

# Part 2

In [4]:
output_dir = './image'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the image
image_path = './src/p5_image.gif'
image = Image.open(image_path)
image_matrix = np.array(image)

# Perform SVD on the image matrix
U, s, Vt = np.linalg.svd(image_matrix, full_matrices=False)

# List of k values for rank k approximations
k_values = [1, 3, 10, 20, 50, 100, 150, 200, 400, 800, 1170]

# Function to normalize the image
def normalize_image(matrix):
    min_val = np.min(matrix)
    max_val = np.max(matrix)
    normalized_matrix = (matrix - min_val) / (max_val - min_val)
    return normalized_matrix

# Recover and save the rank k approximations
for k in k_values:
    # Use the top k singular values and vectors
    S = np.diag(s[:k])
    U_k = U[:, :k]
    Vt_k = Vt[:k, :]
    
    # Reconstruct the image
    image_reconstructed = U_k @ S @ Vt_k
    
    # Normalize the reconstructed image
    image_normalized = normalize_image(image_reconstructed)
    
    # Convert to image and save
    image_save = Image.fromarray(np.uint8(image_normalized * 255))
    image_save.save(os.path.join(output_dir, f'alice_rank_{k}.png'))

print("All rank k approximations have been saved.")

All rank k approximations have been saved.


recovered drawing for k = 150
<br>
<img src="image/alice_rank_150.png" width="400" /> 

# Part 3
We stopped at 1170 because that is the maximum number of singular values for the given 1600 × 1170 image, corresponding to the smaller dimension of the image. In SVD, the number of singular values (and thus the maximum rank of the approximation) is limited by the smaller of the two dimensions of the original matrix (image in this case), which is 1170. This means we cannot have a rank higher than 1170 for this particular image's SVD approximation.

# Part 4

To efficiently store the rank 150 approximation of the image, we need to consider the storage of three components: $U$ matrix, $\Lambda$ diagonal matrix, and the $V^T$ matrix
To calculate the memory required:

- For $U$, we store $1600 \times 150$ units.
- For $\Lambda$, we store $150$ units.
- For $V^T$, we store $150 \times 1170$ units.

total memory of approximation required:
- $\text{Total Memory} = (1600 \times 150) + 150 + (150 \times 1170) = 415650$

Naively saving the image as a matrix of pixel values, where the original image is $1600 \times 1170$. The memory required in that case is:

- $ \text{Naive Storage} = 1600 \times 1170 = 1872000$

To efficiently store the rank $150$ approximation, it requires $415650$ units of memory. In contrast, naively saving the image as a matrix of pixel values requires $1872000$ units of memory. Thus, using the rank $150$ approximation saves $1456350$ units of memory compared to the naive approach.

# Part 5

**High-Frequency Components**: In image, noise and fine details are often high-frequency components. These components are captured by singular vectors corresponding to smaller singular values. Because noise is distributed throughout the image, it doesn't get fully captured until we include a large number of the lower-ranked singular values and vectors.

**Distribution of Singular Values**: The largest singular values capture the most prominent features of the image, such as edges and broad areas of contrast, which are critical for recognizing the main subjects and shapes within the image. As we move to smaller singular values, the details they capture become less about the essential structure of the image and more about subtle variations in texture and color, including noise. The gray haze or noise we observe is due to these subtle variations not being fully captured or separated from the image until we include a large number of singular values in the approximation.

**Nature of Image Noise**: Image noise can arise from various sources, including the original capture process and the inherent texture of the subjects. This noise is not uniformly distributed and can be spatially complex, requiring a high level of detail to accurately represent. Since SVD captures details in order of their energy contribution to the image, the nuanced, less energy-dominant noise patterns only get accurately reconstructed when a high number of singular values and vectors are included.

The persistence of gray haze and random background noise until high ranks in SVD-based image approximations is due to the nature of noise as a high-frequency, low-energy component that is intricately distributed across the image. Accurately capturing and removing this noise requires incorporating a large number of the finer, less significant singular values and vectors into the reconstruction, which only happens at higher ranks.